<a href="https://colab.research.google.com/github/Lee-Minsoo-97/Text-Processing-using-spaCy/blob/main/LA1_Lee_Minsoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LAB Assignment 1

##Minsoo Lee

1234090054\
01/22/2025

In [ ]:
#Library

!pip install spacy==3.5.4
!pip install nltk
!python -m spacy download en_core_web_lg
import spacy
import nltk
nlp = spacy.load("en_core_web_lg")
nltk.download('punkt_tab')# download this package for tokenization
nltk.download('averaged_perceptron_tagger_eng') # download this package for pos tagging
nltk.download('wordnet')
nltk.download('omw-1.4')
from spacy import displacy
nltk.download('maxent_ne_chunker_tab')
nltk.download('words')
!pip install coreferee
!python -m coreferee install en
import coreferee
!pip install dframcy
import spacy
from dframcy import DframCy
import pandas as pd
from collections import defaultdict, Counter
from spacy.matcher import Matcher
import pprint

2025-01-24 20:36:33.595832: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 20:36:33.613338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-24 20:36:33.634298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-24 20:36:33.640689: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 20:36:33.655932: I tensorflow/core/platform/cpu_feature_guar

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker_tab is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


2025-01-24 20:36:57.370321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-24 20:36:57.390923: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-24 20:36:57.397198: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
  Using cached https://github.com/richardpaulhudson/coreferee/raw/master/models/coreferee_model_en.zip (65.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Data import. Show the summary of the input data.
data_path = "/content/drive/MyDrive/Unstructed data/restaurant_reviews_az.csv"
df = pd.read_csv(data_path)
print("Data First 5 Rows")
print(df.head())
print()
print()

print("Summary") #general sturcture of the data
summary = df.describe()
print(summary)
print()
print()

print("Checking NA") #checking NA of the data
print(df.isna().sum())

Data First 5 Rows
                review_id                 user_id             business_id  \
0  IVS7do_HBzroiCiymNdxDg  fdFgZQQYQJeEAshH4lxSfQ  sGy67CpJctjeCWClWqonjA   
1  QP2pSzSqpJTMWOCuUuyXkQ  JBLWSXBTKFvJYYiM-FnCOQ  3w7NRntdQ9h0KwDsksIt5Q   
2  oK0cGYStgDOusZKz9B1qug  2_9fKnXChUjC5xArfF8BLg  OMnPtRGmbY8qH_wIILfYKA   
3  E_ABvFCNVLbfOgRg3Pv1KQ  9MExTQ76GSKhxSWnTS901g  V9XlikTxq0My4gE8LULsjw   
4  Rd222CrrnXkXukR2iWj69g  LPxuausjvDN88uPr-Q4cQA  CA5BOxKRDPGJgdUQ8OUOpw   

   stars  useful  funny  cool  \
0      3       1      1     0   
1      5       1      1     1   
2      5       1      0     0   
3      5       0      0     0   
4      4       1      0     0   

                                                text             date  \
0  OK, the hype about having Hatch chili in your ...  1/27/2020 22:59   
1  Pandemic pit stop to have an ice cream.... onl...   4/19/2020 5:33   
2  I was lucky enough to go to the soft opening a...  2/29/2020 19:43   
3  I've gone to claim Jumper

In [ ]:
# Select the 1 star reviews and 5 star reviews from the dataset.

#Selecing 1 star reviews
one_star_reviews = df[df['stars'] == 1]
one_star_reviews = one_star_reviews[['text']]
print("1 STAR REVIEW")
print(one_star_reviews.head())
print()
print()

#Selecting 5 star reviews
five_star_reviews = df[df['stars'] == 5]
five_star_reviews = five_star_reviews[['text']]
print("5 STAR REVIEW")
print(five_star_reviews.head())

#Selecing 1&5 start reviews
#one_five_star_reviews = pd.concat([one_star_reviews, five_star_reviews])
#one_five_star_reviews = one_five_star_reviews.sample(frac=1).reset_index(drop=True)
#print(one_five_star_reviews.head())


1 STAR REVIEW
                                                 text
5   I stay at the Main Hotel at the Casino from Ju...
13  I don't know what happened to this place but e...
18  Very bad service call for takeout they got our...
20  This place is not worth it. Don't go the food ...
22  I was so looking forward to Del Sols re-openin...


5 STAR REVIEW
                                                text
1  Pandemic pit stop to have an ice cream.... onl...
2  I was lucky enough to go to the soft opening a...
3  I've gone to claim Jumpers all over the US and...
6  In town after a long weekend of hiking and cam...
7  This is the definition of a great family-run b...


### Apply necessary text processing techniques on the selected reviews

## 1. Segmentation/Tokenization

In [ ]:
from spacy.tokens import Doc
# Load spaCy's English model
nlp = spacy.load("en_core_web_lg")

# Tokenize all 1-star reviews
print("Tokenizing all 1-Star Reviews...")
one_star_reviews['tokens'] = one_star_reviews['text'].apply(
    lambda review: [token.text for token in nlp(review)]
)

# Tokenize all 5-star reviews
print("Tokenizing all 5-Star Reviews...")
five_star_reviews['tokens'] = five_star_reviews['text'].apply(
    lambda review: [token.text for token in nlp(review)]
)

# Print head(20) tokenized reviews for 1-star reviews
print("\nTokenized 1-Star Reviews (First 20):")
print(one_star_reviews[['text', 'tokens']].head(20))

print()
print()

# Print head(20) tokenized reviews for 5-star reviews
print("\nTokenized 5-Star Reviews (First 20):")
print(five_star_reviews[['text', 'tokens']].head(20))

Tokenizing all 1-Star Reviews...
Tokenizing all 5-Star Reviews...

Tokenized 1-Star Reviews (First 20):
                                                  text  \
5    I stay at the Main Hotel at the Casino from Ju...   
13   I don't know what happened to this place but e...   
18   Very bad service call for takeout they got our...   
20   This place is not worth it. Don't go the food ...   
22   I was so looking forward to Del Sols re-openin...   
26   I used to love this restaurant but it has stea...   
41   They have the worst customer service skills im...   
51   The staff was extremely rude here. The wait st...   
53   My lord... where do I begin? The name of the c...   
54   Tried to use Priority Pass to pickup a quick c...   
57   I like Taco Bell, or did. Mostly. I have been ...   
75   After picking up from here for delivery severa...   
82   The bathrooms were closed for over half an hou...   
85   If I could give 0 Stars I would we were told 4...   
86   Nope, not again. I al

## 2. Part-of-Speech (POS) Tagging

In [ ]:
# POS Tagging for 1-star reviews
print("Performing POS Tagging for 1-Star Reviews...")
one_star_reviews['pos_tags'] = one_star_reviews['text'].apply(
    lambda review: [(token.text, token.pos_, token.tag_) for token in nlp(review)]
)

# POS Tagging for 5-star reviews
print("Performing POS Tagging for 5-Star Reviews...")
five_star_reviews['pos_tags'] = five_star_reviews['text'].apply(
    lambda review: [(token.text, token.pos_, token.tag_) for token in nlp(review)]
)

Performing POS Tagging for 1-Star Reviews...
Performing POS Tagging for 5-Star Reviews...


In [ ]:
pd.set_option('display.max_colwidth', None)

# Print the first 20 POS-tagged reviews for 1-star reviews
print("\nPOS-Tagged 1-Star Reviews (First 20):")
print(one_star_reviews[['pos_tags']].head(20))

# Print the first 20 POS-tagged reviews for 5-star reviews
print("\nPOS-Tagged 5-Star Reviews (First 20):")
print(five_star_reviews[['pos_tags']].head(20))


POS-Tagged 1-Star Reviews (First 20):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

## 3. Normalization (Lemmatization)

In [ ]:
# Perform lemmatization for 1-star reviews
print("Performing Lemmatization for 1-Star Reviews...")
one_star_reviews['lemmas'] = one_star_reviews['text'].apply(
    lambda review: [token.lemma_ for token in nlp(review)]
)

# Perform lemmatization for 5-star reviews
print("Performing Lemmatization for 5-Star Reviews...")
five_star_reviews['lemmas'] = five_star_reviews['text'].apply(
    lambda review: [token.lemma_ for token in nlp(review)]
)

Performing Lemmatization for 1-Star Reviews...
Performing Lemmatization for 5-Star Reviews...


In [ ]:
# Display the first 20 lemmatized reviews for 1-star
print("\nLemmatized 1-Star Reviews (First 20):")
print(one_star_reviews[['lemmas']].head(20))

print()
print()

# Display the first 20 lemmatized reviews for 5-star
print("\nLemmatized 5-Star Reviews (First 20):")
print(five_star_reviews[['lemmas']].head(20))


Lemmatized 1-Star Reviews (First 20):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          lemmas
5                                                                                                                                                                                                                                                                                                                                      [I, stay,

## 4. Named Entity Recognition (NER)


In [ ]:
from spacy import displacy

# Perform NER processing for all 1-star reviews
print("Performing NER for 1-Star Reviews...")
one_star_reviews['entities'] = one_star_reviews['text'].apply(
    lambda review: [(ent.text, ent.label_, spacy.explain(ent.label_)) for ent in nlp(review).ents]
)

# Perform NER processing for all 5-star reviews
print("Performing NER for 5-Star Reviews...")
five_star_reviews['entities'] = five_star_reviews['text'].apply(
    lambda review: [(ent.text, ent.label_, spacy.explain(ent.label_)) for ent in nlp(review).ents]
)

Performing NER for 1-Star Reviews...
Performing NER for 5-Star Reviews...


In [ ]:
# Display NER visualization for the first 20 1-star reviews
print("\nVisualizing Named Entities for 1-Star Reviews (First 20):")
for idx, review in one_star_reviews['text'].head(20).items():
    print(f"Review {idx + 1}: {review}")
    nlp_doc = nlp(review)  # Process the review with spaCy
    displacy.render(nlp_doc, style='ent', jupyter=True)  # Display visualization
    print("\n")

print()
print("----------"*50)
print()

# Display NER visualization for the first 20 5-star reviews
print("\nVisualizing Named Entities for 5-Star Reviews (First 20):")
for idx, review in five_star_reviews['text'].head(20).items():
    print(f"Review {idx + 1}: {review}")
    nlp_doc = nlp(review)  # Process the review with spaCy
    displacy.render(nlp_doc, style='ent', jupyter=True)  # Display visualization
    print("\n")


Visualizing Named Entities for 1-Star Reviews (First 20):
Review 6: I stay at the Main Hotel at the Casino from July 11 to July 13; it was the WORST experience I've ever had.  For years I have supported this hotel and the casino; however, this time...it was a disaster. Will I go back, hopefullly, NO!




Review 14: I don't know what happened to this place but ever since they moved to a bigger location and expanded to more locations the quality has gone down hill. It's really disappointing because this used to my favorite breakfast place in Tucson. They used to have the best French toast that used to change every month. It was like a cake based french toast it was the best and unique to anywhere else. Now the French toast is just tasteless bread that taste more like overpriced frozen French toast sprinkled with cinnamon and given a fancy name to make it sound special, "churro French toast". The potatoes also used to be the best that were baked with rosemary but now are just deep fried potatoes. It's really disappointing...




Review 19: Very bad service call for takeout they got our order wrong and they wanted to charge me again to go pick up what they forgot very bad service will never go again to any of these restaurants




Review 21: This place is not worth it. Don't go the food wasn't good and the service was horrible




Review 23: I was so looking forward to Del Sols re-opening today and have left thinking i may never go back. All of their machines had been reset, no progressive values, i spent over $600 and barely got a bonus on ANY machine, NO SMOKING ANYWHERE WHILE YOU PLAY, lines were crazy even at ATM, all guests must wear masks and have tempurature check at entry, so many security gaurds i felt uncomfortable like some breach was going to occur, no beverage servers, the machines werent paying out for anyone, and i honestly felt like i was being made to make up for their lost income from being closed. The Gilla River casinos in Chandler were extremely generous, way more comfortable and you could smoke AND PLAY TO WIN, not play to pay casino losses. Very very dissapointed, used to be biggest fan!


...



Review 28: If you want to eat real meat, no preservatives no hormones no junk, but the real thing baby, this is your place to go for a wonderful meal, dessert and fabulous libations! The servings are large- so come hungry!




Review 29: Came here celebrating law school graduation and it was the perfect place. Can't wait to get back soon.




Review 31: Great afternoon on long covered patio facing north , breathtaking catalina Mountain View & railroad train tracks run though too! 
Lucky to get leftover super bowl 54 Sunday specials including bacon jalepeno bacon chowder in a bread bowl along with jalepeno bacon BLT on a hearty bead w side salad . Both were winners in my estimation ! 
Also chosen from researching yelp reviews was the tasty spinach artichoke appetizer served with tortilla chips . Thanks yelpers not disappointed in recommendation, good job!
Sophia our server on point, professional , & fun thank you. 
A cooler sunny Monday afternoon on the patio started getting windy & temps dropping ,I say put it on tape and play it back in a month ! " it's so cool " both Barrio brewery & the weather ..




Review 33: Estrell hotel 

Didn't halve a room Booked but Ismael really looked out and got us a room with no worries. And made our stay a pleasurable. And would do business again. Thank you Ismael for the great hospitality.




Review 34: What a great place to now have in town here! The vibe is modern and comfortable. We were greeted and sat as soon as we came in. We went before 5pm so we were able to take advantage of their amazing drink specials $5 pitchers of beer and $5 glasses of wine. I had rosè and it was so refreshing & crisp, and I loved it. My husbamd got Sippin pretty which was a delicious and fresh tasting sour beer. We ordered prosciutto with brie and the bruschetta board plus 2 additional bruschetta. We got moz tomato and basil, warm artichoke, smoked salmon and pesto, burrata bacon arugula, and sweet n spicy jam w goat cheese. All were aaammmzzzsiiinnggg! Salmon and pesto were our favorite. This is a great spot for couples date night or brunch with friends. I can't wait to return!


## 5. Parsing

In [ ]:
# Perform dependency parsing for all 1-star reviews
print("Performing Dependency Parsing for 1-Star Reviews...")
one_star_reviews['dependency_parse'] = one_star_reviews['text'].apply(
    lambda review: [(token.text, token.dep_, token.head.text) for token in nlp(review)]
)

# Perform dependency parsing for all 5-star reviews
print("Performing Dependency Parsing for 5-Star Reviews...")
five_star_reviews['dependency_parse'] = five_star_reviews['text'].apply(
    lambda review: [(token.text, token.dep_, token.head.text) for token in nlp(review)]
)

Performing Dependency Parsing for 1-Star Reviews...
Performing Dependency Parsing for 5-Star Reviews...


In [ ]:
# Display Dependency Parsing Visualization for 1-star reviews (First 20)
print("\nDependency Parsing Visualization for 1-Star Reviews (First 20):")
for idx, review in one_star_reviews['text'].head(20).items():
    print(f"Review {idx + 1}: {review}")
    nlp_doc = nlp(review)
    displacy.render(nlp_doc, style='dep', jupyter=True)  # Dependency parsing visualization
    print("\n")

print()
print("----------"*50)
print()


# Display Dependency Parsing Visualization for 5-star reviews (First 20)
print("\nDependency Parsing Visualization for 5-Star Reviews (First 20):")
for idx, review in five_star_reviews['text'].head(20).items():
    print(f"Review {idx + 1}: {review}")
    nlp_doc = nlp(review)
    displacy.render(nlp_doc, style='dep', jupyter=True)  # Dependency parsing visualization
    print("\n")


Dependency Parsing Visualization for 1-Star Reviews (First 20):
Review 6: I stay at the Main Hotel at the Casino from July 11 to July 13; it was the WORST experience I've ever had.  For years I have supported this hotel and the casino; however, this time...it was a disaster. Will I go back, hopefullly, NO!




Review 14: I don't know what happened to this place but ever since they moved to a bigger location and expanded to more locations the quality has gone down hill. It's really disappointing because this used to my favorite breakfast place in Tucson. They used to have the best French toast that used to change every month. It was like a cake based french toast it was the best and unique to anywhere else. Now the French toast is just tasteless bread that taste more like overpriced frozen French toast sprinkled with cinnamon and given a fancy name to make it sound special, "churro French toast". The potatoes also used to be the best that were baked with rosemary but now are just deep fried potatoes. It's really disappointing...




Review 19: Very bad service call for takeout they got our order wrong and they wanted to charge me again to go pick up what they forgot very bad service will never go again to any of these restaurants




Review 21: This place is not worth it. Don't go the food wasn't good and the service was horrible




Review 23: I was so looking forward to Del Sols re-opening today and have left thinking i may never go back. All of their machines had been reset, no progressive values, i spent over $600 and barely got a bonus on ANY machine, NO SMOKING ANYWHERE WHILE YOU PLAY, lines were crazy even at ATM, all guests must wear masks and have tempurature check at entry, so many security gaurds i felt uncomfortable like some breach was going to occur, no beverage servers, the machines werent paying out for anyone, and i honestly felt like i was being made to make up for their lost income from being closed. The Gilla River casinos in Chandler were extremely generous, way more comfortable and you could smoke AND PLAY TO WIN, not play to pay casino losses. Very very dissapointed, used to be biggest fan!


...



Review 28: If you want to eat real meat, no preservatives no hormones no junk, but the real thing baby, this is your place to go for a wonderful meal, dessert and fabulous libations! The servings are large- so come hungry!




Review 29: Came here celebrating law school graduation and it was the perfect place. Can't wait to get back soon.




Review 31: Great afternoon on long covered patio facing north , breathtaking catalina Mountain View & railroad train tracks run though too! 
Lucky to get leftover super bowl 54 Sunday specials including bacon jalepeno bacon chowder in a bread bowl along with jalepeno bacon BLT on a hearty bead w side salad . Both were winners in my estimation ! 
Also chosen from researching yelp reviews was the tasty spinach artichoke appetizer served with tortilla chips . Thanks yelpers not disappointed in recommendation, good job!
Sophia our server on point, professional , & fun thank you. 
A cooler sunny Monday afternoon on the patio started getting windy & temps dropping ,I say put it on tape and play it back in a month ! " it's so cool " both Barrio brewery & the weather ..




Review 33: Estrell hotel 

Didn't halve a room Booked but Ismael really looked out and got us a room with no worries. And made our stay a pleasurable. And would do business again. Thank you Ismael for the great hospitality.




Review 34: What a great place to now have in town here! The vibe is modern and comfortable. We were greeted and sat as soon as we came in. We went before 5pm so we were able to take advantage of their amazing drink specials $5 pitchers of beer and $5 glasses of wine. I had rosè and it was so refreshing & crisp, and I loved it. My husbamd got Sippin pretty which was a delicious and fresh tasting sour beer. We ordered prosciutto with brie and the bruschetta board plus 2 additional bruschetta. We got moz tomato and basil, warm artichoke, smoked salmon and pesto, burrata bacon arugula, and sweet n spicy jam w goat cheese. All were aaammmzzzsiiinnggg! Salmon and pesto were our favorite. This is a great spot for couples date night or brunch with friends. I can't wait to return!


##  Top 20 frequently used nouns, verbs, adjectives, and names in 1 star reviews



In [ ]:
# Extract all tokens for 1-star reviews
print("Extracting parts of speech for 1-star reviews...")

def extract_pos(tokens, pos_type):
    """Helper function to filter tokens by part-of-speech."""
    return [token.text.lower() for token in nlp(tokens) if token.pos_ == pos_type]

# Collect nouns, verbs, adjectives, and names
nouns = []
verbs = []
adjectives = []
names = []

for review in one_star_reviews['text']:
    doc = nlp(review)
    nouns.extend([token.text.lower() for token in doc if token.pos_ == "NOUN"])
    verbs.extend([token.text.lower() for token in doc if token.pos_ == "VERB"])
    adjectives.extend([token.text.lower() for token in doc if token.pos_ == "ADJ"])
    names.extend([ent.text.lower() for ent in doc.ents if ent.label_ == "PERSON"])


Extracting parts of speech for 1-star reviews...


In [ ]:
# Get the top 20 most common nouns, verbs, adjectives, and names
top_nouns = Counter(nouns).most_common(20)
top_verbs = Counter(verbs).most_common(20)
top_adjectives = Counter(adjectives).most_common(20)
top_names = Counter(names).most_common(20)

# Print results
print("\nTop 20 Nouns:")
print(top_nouns)


Top 20 Nouns:
[('food', 6490), ('order', 3786), ('time', 3393), ('service', 3023), ('place', 2959), ('minutes', 2383), ('restaurant', 2021), ('manager', 1522), ('people', 1433), ('customer', 1307), ('chicken', 1265), ('table', 1211), ('location', 1124), ('experience', 1046), ('pizza', 1032), ('staff', 1022), ('way', 947), ('times', 933), ('server', 926), ('meal', 873)]


In [ ]:
print("\nTop 20 Verbs:")
print(top_verbs)


Top 20 Verbs:
[('had', 4060), ('get', 3019), ('ordered', 2858), ('have', 2594), ('go', 2558), ('said', 2381), ('got', 2231), ('asked', 2025), ('told', 1912), ('went', 1700), ('came', 1653), ('going', 1319), ('know', 1265), ('come', 1190), ('give', 1175), ('eat', 1132), ('took', 1115), ('take', 1106), ('make', 1053), ('waited', 1038)]


In [ ]:
print("\nTop 20 Adjectives:")
print(top_adjectives)


Top 20 Adjectives:
[('good', 1859), ('other', 1394), ('bad', 1223), ('rude', 1000), ('worst', 954), ('last', 918), ('cold', 888), ('more', 853), ('first', 848), ('terrible', 772), ('horrible', 770), ('great', 724), ('better', 684), ('disappointed', 681), ('wrong', 634), ('same', 631), ('sure', 620), ('new', 572), ('many', 555), ('small', 507)]


In [ ]:
print("\nTop 20 Names:")
print(top_names)


Top 20 Names:
[('denny', 48), ('mac', 47), ('luke', 28), ('carne asada', 28), ('chimi', 26), ('bland', 25), ('lo mein', 24), ('christina', 21), ('jack', 20), ('rice', 20), ('yuck', 18), ('wendy', 17), ('grant', 16), ('ina', 16), ('sonoran', 16), ('waiter', 16), ('said', 15), ('an', 15), ('popeye', 15), ('doordash', 14)]


##  Top 20 frequently used nouns, verbs, adjectives, and names in 5 star reviews


In [ ]:
# Extract all tokens for 5-star reviews
print("Extracting parts of speech for 5-star reviews...")

# Collect nouns, verbs, adjectives, and names
nouns = []
verbs = []
adjectives = []
names = []

for review in five_star_reviews['text']:
    doc = nlp(review)
    nouns.extend([token.text.lower() for token in doc if token.pos_ == "NOUN"])
    verbs.extend([token.text.lower() for token in doc if token.pos_ == "VERB"])
    adjectives.extend([token.text.lower() for token in doc if token.pos_ == "ADJ"])
    names.extend([ent.text.lower() for ent in doc.ents if ent.label_ == "PERSON"])

# Get the top 20 most common nouns, verbs, adjectives, and names
top_nouns = Counter(nouns).most_common(20)
top_verbs = Counter(verbs).most_common(20)
top_adjectives = Counter(adjectives).most_common(20)
top_names = Counter(names).most_common(20)


Extracting parts of speech for 5-star reviews...


In [ ]:
# Print results
print("\nTop 20 Nouns in 5-Star Reviews:")
print(top_nouns)


Top 20 Nouns in 5-Star Reviews:
[('food', 15525), ('place', 9735), ('service', 7229), ('time', 5573), ('staff', 4057), ('restaurant', 4000), ('menu', 3171), ('chicken', 2871), ('pizza', 2468), ('experience', 2121), ('order', 2004), ('sauce', 1933), ('meal', 1911), ('breakfast', 1885), ('lunch', 1824), ('salad', 1784), ('spot', 1750), ('family', 1696), ('tacos', 1692), ('cheese', 1666)]


In [ ]:
print("\nTop 20 Verbs in 5-Star Reviews:")
print(top_verbs)


Top 20 Verbs in 5-Star Reviews:
[('had', 10959), ('have', 5814), ('go', 4429), ('love', 4030), ('get', 3877), ('ordered', 3852), ('got', 3590), ('try', 3372), ('recommend', 3225), ('made', 2670), ('come', 2360), ('came', 2189), ('eat', 2062), ('make', 1872), ('tried', 1807), ('went', 1802), ('take', 1744), ('loved', 1730), ('has', 1654), ('order', 1612)]


In [ ]:
print("\nTop 20 Adjectives in 5-Star Reviews:")
print(top_adjectives)


Top 20 Adjectives in 5-Star Reviews:
[('great', 12284), ('good', 9536), ('delicious', 6397), ('amazing', 5673), ('best', 5209), ('friendly', 4365), ('fresh', 3383), ('nice', 3206), ('excellent', 2534), ('favorite', 2149), ('first', 2108), ('more', 2052), ('other', 1984), ('perfect', 1969), ('little', 1955), ('hot', 1742), ('awesome', 1696), ('wonderful', 1634), ('new', 1589), ('tasty', 1564)]


In [ ]:
print("\nTop 20 Names in 5-Star Reviews:")
print(top_names)


Top 20 Names in 5-Star Reviews:
[('sonoran', 251), ('mac', 147), ('benedict', 127), ('carne asada', 101), ('hubby', 74), ('al', 64), ('tumerico', 61), ('lo mein', 56), ('joe', 44), ('john', 42), ('daniel', 42), ('mary', 40), ('rocco', 39), ('grant', 39), ('noodleholics', 39), ('bobo', 38), ('rollie', 37), ('postino', 35), ('vivace', 35), ('boba', 35)]


## Observation on the language of 1 star and 5 star reviews. Key to a good restaurant experience?


Key Observations on 1-Star vs. 5-Star Reviews

Differences in Language

	1.	Food Quality:
	•	Positive reviews emphasize “delicious,” “fresh,” and “tasty,” while negative reviews mention “cold,” “bad,” and “wrong.”

	2.	Service:
	•	5-star reviews highlight “friendly,” “amazing,” and “great” service.
	•	1-star reviews focus on “rude,” “worst,” and “bad” service, with complaints about delays.

	3.	Ambiance:
	•	Good reviews mention “spot,” “family,” and “experience,” showing the value of a welcoming atmosphere.
	•	Negative reviews imply issues with cleanliness and seating.

	4.	Staff Behavior:
	•	Staff attitude significantly impacts reviews, with appreciation for professionalism in 5-star reviews and dissatisfaction with “manager” and “server” interactions in 1-star reviews.

	5.	Consistency:
	•	Frequent mentions of “times” and “more” suggest that consistent quality builds loyalty, while inconsistencies cause frustration.







 **- I have worked with ChatGPT.**

In [ ]:
#html converstion

!pip install jupyter
!pip install nbconvert

[NbConvertApp] Converting notebook /content/drive/MyDrive/Unstructed data/LA1_Lee_Minsoo.ipynb to html
[NbConvertApp] Writing 617905 bytes to /content/drive/MyDrive/Unstructed data/LA1_Lee_Minsoo.html


In [ ]:
!jupyter nbconvert '/content/drive/MyDrive/Unstructed data/LA1_Lee_Minsoo.ipynb' --to html

[NbConvertApp] Converting notebook /content/drive/MyDrive/Unstructed data/LA1_Lee_Minsoo.ipynb to html
[NbConvertApp] Writing 604914 bytes to /content/drive/MyDrive/Unstructed data/LA1_Lee_Minsoo.html
